In [11]:
import jax
from jax import jit
import jax.numpy as jnp
import flax.linen as nn
import graphviz

In [12]:
class SubModel(nn.Module):
    feature_size: int
    
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(self.feature_size)(x)
        return x
    
class MainModel(nn.Module):
    feature_size: int = 2
    
    @nn.compact
    def __call__(self, x1, x2):
        x = jnp.concatenate((x1, x2), axis=-1)
        return nn.Dense(self.feature_size)(x)

    
submodel1 = SubModel(5)
submodel2 = SubModel(3)
mainmodel = MainModel()


# Dummy data to initialize
b1 = jnp.ones((1, 10))
b2 = jnp.ones((1, 5))

# Initalize the model parameters
p1 = submodel1.init(jax.random.PRNGKey(0), b1) #init dense layer
p2 = submodel2.init(jax.random.PRNGKey(0), b2) #init dense layer



#init dense layer - just for the shape
p3 = mainmodel.init(jax.random.PRNGKey(0), jnp.ones((1, 5)), jnp.ones((1, 3))) 






def main(x1, x2, p1, p2, pmain):
    y1 = submodel1.apply(p1, x1)
    y2 = submodel2.apply(p2, x2)
    return mainmodel.apply(pmain, y1, y2)

lowered = jax.jit(main).lower(b1, b2, p1, p2, p3)


# # If you want to see the IR
# cir = lowered.compiler_ir()
# cir.dump()


# view the computation graph
comp_dot = graphviz.Source(lowered._xla_computation().as_hlo_dot_graph())
comp_dot 

In [24]:
import tensorflow as tf

@tf.function(jit_compile=True)
def not_compilable(x):
  return tf.unique(x)


@tf.function(jit_compile=True)
def recompiled_on_launch(a, b):
  return a + b

recompiled_on_launch(tf.ones([1, 10]), tf.ones([1, 10]))
recompiled_on_launch(tf.ones([1, 100]), tf.ones([1, 100]))


not_compilable(tf.ones([10]))


tf.unique(tf.ones([10]))

tf.ones([10])


tf.unique([1,2,3,])

Unique(y=<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>, idx=<tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 1, 2], dtype=int32)>)